In [1]:
# Imports
import argparse         # parser for command-line options
import getopt           # variable-length params
import sys              # argv
import time             # time handling
import numpy

from collections import defaultdict

In [2]:
#Define variables real-deal attempt
items = numpy.genfromtxt('list1718_cleaner.csv',delimiter=";",
                         dtype=None,names=['name','club','pos','price','value','grade'])
weight = items['price']
value = items['value']
positions = items['pos']

#Capacities for different positions
goalkeeper_range = numpy.array([4.5, 4.2, 4.0, 3.7, 3.6, 3.5, 3.2,
                                2.8, 2.4, 2.2, 1.8, 1.0, 0.8, 0.7,
                                0.6, 0.5, 0.4, 0.3, 0.2])
defender_range = numpy.arange(0.8,17.3,0.1)
midfielder_range = numpy.arange(1.0,39.1,0.1)
striker_range = numpy.arange(0.6,23.4,0.1)

#Multiple capacities/weights by 10 to convert to integer values
items['price']=10*items['price']
goalkeeper_range=10*goalkeeper_range
defender_range=10*defender_range
midfielder_range=10*midfielder_range
striker_range=10*striker_range

#Maximum capacities per position
goalkeeper_maxcap = max(goalkeeper_range)
defender_maxcap = max(defender_range)
midfielder_maxcap=max(midfielder_range)
striker_maxcap=max(striker_range)

#Maximum picks per position
goalkeeper_maxitem = 1
defender_maxitem = 4
midfielder_maxitem = 5 
striker_maxitem = 3


#print positions #MIT, STU, TOR, ABW

for i in range(items.size):
    if positions[i]=='MIT': 
        if 'MITarray' in dir():
            MITarray=numpy.append(MITarray,items[i])
        else:
            MITarray=items[i]
    elif positions[i]=='STU':
        if 'STUarray' in dir():
            STUarray=numpy.append(STUarray,items[i])
        else:
            STUarray=items[i]
    elif positions[i]=='ABW':
        if 'ABWarray' in dir():
            ABWarray=numpy.append(ABWarray,items[i])
        else:
            ABWarray=items[i]
    elif positions[i]=='TOR':
        if 'TORarray' in dir():
            TORarray=numpy.append(TORarray,items[i])
        else:
            TORarray=items[i]

In [3]:
#define the solver as function to use multiple times

def solve3dmod(capacity,value,weight,maxitems):
    """
    solve the 3d-knapsack problem specified in its parameters: capacity is the
    overall capacity of the knapsack and the ith position of the arrays value
    and weight specify the value and weight of the ith item. This is called the
    3d-knapsack not because it refers to a cuboid but because it also considers
    a maximum number of items to insert which is given in the last parameter
    """

    print
    print " Solving 3d ..."
    print
    
    # initialization - the number of items is taken from the length of any array
    # which shall have the same length
    nbitems = len (value)

    # we use dynamic programming to solve this problem. Thus, we'll need a table
    # that contains (N, M) cells where 0<=N<=capacity and 0<=M<=nbitems
    table=dict ()

    # initialize the contents of the dictionary for all capacities and number of
    # items to another dictionary which returns 0 by default
    for icapacity in range (0, 1+capacity):
        table [icapacity]=dict ()
        for items in range (0, 1+nbitems):
            table [icapacity][items] = defaultdict (int)

    # now we are ready to start, ... for the first j items
    for j in range (0, nbitems):
            # for all capacities ranging from 1 to the maximum overall capacity
        for i in range (1, 1+capacity):
                # and for all cardinalities of items from 1 until the maximum
                # allowed
            for k in range (1, 1+maxitems):
                    # if this item can not be inserted
                if (weight [j] > i):
                        table [i][1+j][k] = table [i][j][k]   # just do not add it

                    # otherwise, consider inserting it
                else:

                        # if this is item is known to fit the knapsack (because its
                        # weight does not exceed the current capacity and adding it
                        # creates a set with a cardinality less or equal than the
                        # cardinality currently considered), then compute the
                        # optimal value as usual but considering sets of the same
                        # cardinality (k)
                    if (j<k):
                        table [i][1+j][k] = max (table[i][j][k],table[i-weight[j]][j][k]+value[j])

                    else:
                        prev = []

                            # retrieve the optimal solution for all values of
                            # (i-weight[j], kappa [0 .. j-1], k-1)
                        for kappa in range (0, 1+j):
                            prev.append (table[i-weight[j]][kappa][k-1])

                            # and consider inserting this item taking into account
                            # the optimal solution from the preceding cardinality
                            # considering all sets of items
                            table [i][1+j][k] = max (table[i][j][k],
                                                     max (prev) + value[j])
        
    #return table
    # show the header
    print
    print " Overall capacity: %i" % capacity
    print " # of items: %i" % nbitems
    print " maximum number of items: %i" % maxitems
    print " Value of the items: "
    print "\t ",
    print value
    print " Weight of the items: "
    print "\t ",
    print weight
    print

    # and now, for all cardinalities of the items
    #for k in range (0, 1+maxitems):
    k = maxitems

    print " Maximum number of items: %i" % k

    # and all capacities ranging from zero to the maximum overall capacity
    for i in range (0, 1+capacity):

        # and all sets of items, from 0 to the maximum number of items
        for j in range (0, 1+nbitems):

            print "%4d " % table[i][j][k],

        print

    print
    print



## Defender table

In [9]:
solve3dmod(int(defender_maxcap),ABWarray['value'],ABWarray['price'],defender_maxitem)


 Solving 3d ...


 Overall capacity: 171
 # of items: 169
 maximum number of items: 4
 Value of the items: 
	  [127 111 111  40  65 117  57 142 101  99 100  48  95  49  76 122  43  83
 127  90  83  82  70 115  39  69  93  96  68  83  42 102  81  92  92  68
  13  -1  -1  94  89   8  17  50  56  47  73  59  -1  -1  60  70  26  -1
  -1  47  54  41  45   7  64  75  41  40  -1  -1  39  61  45  34  -1  46
   1  24  30  48  23   9  29  -1  30  35  21  15  -1  16  36  29  -1  -1
  -1  -1  -1  27   5   9  10   9   0   1   2  19   4   9  -1  -1  21   0
  -1   1   7  -1   3   9  15  -5  26  36  16  -1   0  -1  -1   5  -7  11
  13  26   4   7   1  25 -23  -1  -1   3   0  -1  -1   0  -1   1  -1  -1
   0  -3  -3   0  -1  -1   0  -1   0  -1  -1  -1   0   6   2   0   0  -1
  -1   0  -1   0   0   0   0]
 Weight of the items: 
	  [ 45.  43.  43.  42.  42.  40.  36.  36.  35.  35.  35.  32.  32.  32.  30.
  30.  30.  30.  30.  30.  28.  28.  28.  28.  28.  28.  27.  27.  26.  26.
  26.  26.  26.  26.  2

   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     3     6     8     8     8     8     8     8     8     8

   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    60    70    70    70    70    70    70    70    70    70    70    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    75    81    81    81    81    81    81    81    81    81

   0     0     0     0     0     0     0    57   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142

   0   127   127   127   127   127   127   127   142   142   142   142   142   142   142   142   142   142   142   142   142   142   165   165   198   198   198   208   211   211   211   211   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229   229

...

   0   127   238   238   238   238   244   244   269   269   269   269   291   338   338   338   365   365   365   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   391   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   403   408   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419   419

   0   127   238   238   238   238   244   244   326   370   370   370   370   370   370   370   391   391   391   396   396   396   414   414   447   447   447   457   460   460   460   460   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466   466

   0   127   238   349   349   349   355   355   386   386   386   386   386   386   386   386   441   441   448   492   492   492   492   492   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506   506

   0   127   238   349   349   349   355   355   386   417   459   460   460   465   465   465   492   492   492   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518

   0   127   238   349   349   349   355   355   443   487   487   487   487   487   487   487   508   508   508   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518   518

## Midfielder table 

In [10]:
solve3dmod(int(midfielder_maxcap),MITarray['value'],MITarray['price'],midfielder_maxitem)


 Solving 3d ...


 Overall capacity: 390
 # of items: 217
 maximum number of items: 5
 Value of the items: 
	  [227  -1 172 216 171 199 128  94  75 117 159  33  95  79 123  78  -1 169
  66 137 114 161 124 102  48  67 127  28  53  36  77  68  65  -1  59  67
  44  99  -1  85  75  -1  27  78  96  62  83  72  71  23   8  15  34  -1
  68   1  72  15  13  -1  88  62  33   5  -1  -1  87  63  42  79  10  -1
  -1  38  11  34  28  47  56  -1  58  47  -1 -26  -1  31  36  22  58  -8
  -7  -1  29  38   6  14  32  -8  53  41  39  -1  -3  21  15  18  -1  40
  -1   5  37  12 -11  -1   1  18  -1  29  -1  29   2  13  68  -1  28  11
  -4  -1   2  24  -1  30   6 -20  22 -24  13  10   0  -1   9  24  27  -1
  18  -1   4   0  16  -5  -1  -1  -1  20  19  22   8  -1  -1   2  -1   1
   0   4  18   5   7  -4  -4  -1   7  -1  -1   0   0  -1   4  -1  -3   0
  15   8  -1   0   4 -10  -1   1   0   1   1  -1  -1  -1 -17  -1   0   1
  -1   2  -1   0   0  -1   0  -1   0   0  -1   0   0   0   1  -1   0  -1
   0]
 Weigh

   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    21    21    21    21    40    40    40    40    40    40    40    40    40    40    40    40    40    40    40    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68    68

   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    71    71    71    71    71    71    71    71    72    72    72    72    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88    88

   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   124   124   124   124   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127   127

...

   0   227   227   399   615   786   814   814   814   814   814   814   814   841   841   924   924   924   970   970   970   970   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972

   0   227   227   399   615   786   814   814   814   814   814   819   834   896   896   924   924   924   970   970   970   970   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972   972

   0   227   227   399   615   786   814   814   814   814   814   918   918   918   918   937   937   937   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983

   0   227   227   399   615   786   814   814   814   814   853   973   973   973   973   973   973   973   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983

   0   227   227   399   615   786   814   886   886   886   887   973   973   973   973   973   973   973   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983   983

## Striker table

In [11]:
solve3dmod(int(striker_maxcap),STUarray['value'],STUarray['price'],striker_maxitem)


 Solving 3d ...


 Overall capacity: 232
 # of items: 91
 maximum number of items: 3
 Value of the items: 
	  [291 215 236 187  96 142 175 142  77  46  86  23  42 111 114 136  87 107
 137 110  46  24  -1  -1  84 119  67  -1  22  48  77  67  -1  42  -1  73
  83  -5  48  -1  -1  -1  31  -1  27  -6   7  -1  -1  12  37  43   0   8
 -15  -7   3  16   0   2   3   6  -1  -7  27   0   0  -1   0   7  -2  -1
  -1   2   0  -1  -1  -1  -3  -1  -1  -1   0   1  -1  -1  -1   0  -1  -1
  -1]
 Weight of the items: 
	  [ 85.  85.  64.  60.  55.  55.  55.  48.  48.  45.  42.  42.  35.  34.  34.
  32.  32.  30.  30.  30.  28.  28.  28.  27.  27.  27.  27.  27.  26.  25.
  25.  25.  25.  24.  24.  24.  24.  24.  24.  23.  22.  20.  20.  20.  20.
  18.  18.  18.  17.  16.  16.  15.  15.  15.  14.  14.  14.  12.  12.  10.
  10.  10.  10.   8.   8.   8.   7.   6.   6.   6.   6.   6.   5.   5.   5.
   5.   4.   4.   3.   3.   3.   2.   2.   2.   2.   2.   2.   2.   2.   2.
   2.]

 Maximum number of items: 3


   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0    12    37    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    44    44    44    44    44    44    44    44 
   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     7     7     7    12    37    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43    43

   0     0     0     0     0     0     0     0   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   150   160   160   160   160   160   160   160   160   160   160   160   160   160   160   174   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   180   181   181   181   181   181   181   181   181 
   0     0     0     0     0     0     0     0   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   142   144   144   144   144   150   160   160   160   160   160   160   168   168   168   168   168   168   168   168   174   180   180   180   180   180   180   180   180   180   180   180   180   180   189   189   189   189   189   189   189   189   189   189

   0     0     0   236   236   236   236   236   236   236   236   236   236   236   236   236   278   278   278   279   279   279   279   279   279   279   279   279   279   279   279   280   281   281   298   298   329   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339 
   0     0     0   236   236   236   236   236   236   236   236   236   236   236   253   256   278   278   278   279   279   279   279   279   279   279   294   294   294   294   304   333   333   333   333   333   333   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339   339

   0   291   291   291   291   291   291   317   378   378   378   378   378   378   378   378   392   392   392   415   415   415   415   415   415   415   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431   431 
   0   291   291   291   291   291   329   362   378   378   378   378   378   378   378   378   392   392   398   428   428   428   428   428   428   428   431   431   431   431   431   431   431   431   431   431   431   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438   438

   0   291   291   291   478   478   478   478   478   478   478   478   478   478   489   492   514   514   534   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564 
   0   291   291   291   478   478   478   478   478   478   478   478   478   478   489   492   514   514   534   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564   564

   0   291   506   527   527   527   527   553   565   565   565   565   565   565   577   580   614   614   614   615   615   615   615   615   615   615   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646 
   0   291   506   527   527   527   527   553   565   565   565   565   565   565   577   580   614   614   614   615   615   615   615   615   615   615   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646   646

   0   291   506   527   527   623   669   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702 
   0   291   506   527   527   623   669   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702   702

## Goalkeeper table (trivial case)

In [12]:
solve3dmod(int(goalkeeper_maxcap),TORarray['value'],TORarray['price'],goalkeeper_maxitem)


 Solving 3d ...


 Overall capacity: 45
 # of items: 59
 maximum number of items: 1
 Value of the items: 
	  [147 187 175 191 204 155 162 153 161 129 190  86 164 102  -1 153  74  -1
  -1  -1  -1  -1  14   0  22   0   0  24  51   0   0  26   0   0   0  -1
   2   0   0  16   8  -1  -1  -1   0  -1   0   6   0  -1   0   0  -1   0
   0  -1  -1   0  -1]
 Weight of the items: 
	  [ 45.  42.  40.  40.  40.  37.  37.  36.  36.  35.  35.  32.  32.  28.  24.
  22.  22.  22.  18.  18.  18.  18.  10.  10.  10.  10.  10.  10.   8.   8.
   8.   8.   7.   6.   6.   6.   6.   5.   5.   4.   4.   3.   3.   3.   2.
   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.]

 Maximum number of items: 1
   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  

## Odds and ends used in arriving at the above code

In [34]:
# Define variables dummy
items = numpy.genfromtxt ('dummy.csv', delimiter=";", dtype=None, names=['name','club','pos','price','value'])
weight = items['price']

value = items['value']
capacity = 170
maxitems = 2

solve3dmod(capacity,value,weight,maxitems)


 Solving 3d ...


 Overall capacity: 170
 # of items: 6
 maximum number of items: 2
 Value of the items: 
	  [ 10 200 100  40 120  50]
 Weight of the items: 
	  [85 85 85 85 50 50]

 Maximum number of items: 2
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     

# Solve 3d

In [13]:
# initialization - the number of items is taken from the length of any array
# which shall have the same length
nbitems = len (value)

# we use dynamic programming to solve this problem. Thus, we'll need a table
# that contains (N, M) cells where 0<=N<=capacity and 0<=M<=nbitems
table=dict ()

# initialize the contents of the dictionary for all capacities and number of
# items to another dictionary which returns 0 by default
for icapacity in range (0, 1+capacity):
    table [icapacity]=dict ()
    for items in range (0, 1+nbitems):
        table [icapacity][items] = defaultdict (int)

# now we are ready to start, ... for the first j items
for j in range (0, nbitems):
        # for all capacities ranging from 1 to the maximum overall capacity
    for i in range (1, 1+capacity):
            # and for all cardinalities of items from 1 until the maximum
            # allowed
        for k in range (1, 1+maxitems):
                # if this item can not be inserted
            if (weight [j] > i):
                    table [i][1+j][k] = table [i][j][k]   # just do not add it
                    
                # otherwise, consider inserting it
            else:
                    
                    # if this is item is known to fit the knapsack (because its
                    # weight does not exceed the current capacity and adding it
                    # creates a set with a cardinality less or equal than the
                    # cardinality currently considered), then compute the
                    # optimal value as usual but considering sets of the same
                    # cardinality (k)
                if (j<k):
                    table [i][1+j][k] = max (table[i][j][k],table[i-weight[j]][j][k]+value[j])

                else:
                    prev = []

                        # retrieve the optimal solution for all values of
                        # (i-weight[j], kappa [0 .. j-1], k-1)
                    for kappa in range (0, 1+j):
                        prev.append (table[i-weight[j]][kappa][k-1])

                        # and consider inserting this item taking into account
                        # the optimal solution from the preceding cardinality
                        # considering all sets of items
                        table [i][1+j][k] = max (table[i][j][k],
                                                 max (prev) + value[j])
                    

# show table

In [19]:
    # show the header
    print
    print " Overall capacity: %i" % capacity
    print " # of items: %i" % nbitems
    print " maximum number of items: %i" % maxitems
    print " Value of the items: "
    print "\t ",
    print value
    print " Weight of the items: "
    print "\t ",
    print weight
    print

    # and now, for all cardinalities of the items
    #for k in range (0, 1+maxitems):
    k = maxitems

    print " Maximum number of items: %i" % k

    # and all capacities ranging from zero to the maximum overall capacity
    for i in range (0, 1+capacity):

        # and all sets of items, from 0 to the maximum number of items
        for j in range (0, 1+nbitems):

            print "%4d " % table[i][j][k],

        print

    print
    print




 Overall capacity: 170
 # of items: 6
 maximum number of items: 2
 Value of the items: 
	  [ 10 200 100  40 120  50]
 Weight of the items: 
	  [85 85 85 85 50 50]

 Maximum number of items: 2
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     0     0     0     0     0     0 
   0     

In [ ]:
def solve3dmod(capacity,value,weight,maxitems):
    """
    solve the 3d-knapsack problem specified in its parameters: capacity is the
    overall capacity of the knapsack and the ith position of the arrays value
    and weight specify the value and weight of the ith item. This is called the
    3d-knapsack not because it refers to a cuboid but because it also considers
    a maximum number of items to insert which is given in the last parameter
    """

    print
    print " Solving 3d ..."
    print
    
    # initialization - the number of items is taken from the length of any array
    # which shall have the same length
    nbitems = len (value)

    # we use dynamic programming to solve this problem. Thus, we'll need a table
    # that contains (N, M) cells where 0<=N<=capacity and 0<=M<=nbitems
    table=dict ()

    # initialize the contents of the dictionary for all capacities and number of
    # items to another dictionary which returns 0 by default
    for icapacity in range (0, 1+capacity):
        table [icapacity]=dict ()
        for items in range (0, 1+nbitems):
            table [icapacity][items] = defaultdict (int)

    # now we are ready to start, ... for the first j items
    for j in range (0, nbitems):
            # for all capacities ranging from 1 to the maximum overall capacity
        for i in range (1, 1+capacity):
                # and for all cardinalities of items from 1 until the maximum
                # allowed
            for k in range (1, 1+maxitems):
                    # if this item can not be inserted
                if (weight [j] > i):
                        table [i][1+j][k] = table [i][j][k]   # just do not add it

                    # otherwise, consider inserting it
                else:

                        # if this is item is known to fit the knapsack (because its
                        # weight does not exceed the current capacity and adding it
                        # creates a set with a cardinality less or equal than the
                        # cardinality currently considered), then compute the
                        # optimal value as usual but considering sets of the same
                        # cardinality (k)
                    if (j<k):
                        table [i][1+j][k] = max (table[i][j][k],table[i-weight[j]][j][k]+value[j])

                    else:
                        prev = []

                            # retrieve the optimal solution for all values of
                            # (i-weight[j], kappa [0 .. j-1], k-1)
                        for kappa in range (0, 1+j):
                            prev.append (table[i-weight[j]][kappa][k-1])

                            # and consider inserting this item taking into account
                            # the optimal solution from the preceding cardinality
                            # considering all sets of items
                            table [i][1+j][k] = max (table[i][j][k],
                                                     max (prev) + value[j])
        
    #return table
    # show the header
    print
    print " Overall capacity: %i" % capacity
    print " # of items: %i" % nbitems
    print " maximum number of items: %i" % maxitems
    print " Value of the items: "
    print "\t ",
    print value
    print " Weight of the items: "
    print "\t ",
    print weight
    print

    # and now, for all cardinalities of the items
    #for k in range (0, 1+maxitems):
    k = maxitems

    print " Maximum number of items: %i" % k

    # and all capacities ranging from zero to the maximum overall capacity
    for i in range (0, 1+capacity):

        # and all sets of items, from 0 to the maximum number of items
        for j in range (0, 1+nbitems):

            print "%4d " % table[i][j][k],

        print

    print
    print

# show the header
    print
    print " Overall capacity: %i" % capacity
    print " # of items: %i" % nbitems
    print " maximum number of items: %i" % maxitems
    print " Value of the items: "
    print "\t ",
    print value
    print " Weight of the items: "
    print "\t ",
    print weight
    print

    # and now, for all cardinalities of the items
    #for k in range (0, 1+maxitems):
    k = maxitems

    print " Maximum number of items: %i" % k

    # and all capacities ranging from zero to the maximum overall capacity
    for i in range (0, 1+capacity):

        # and all sets of items, from 0 to the maximum number of items
        for j in range (0, 1+nbitems):

            print "%4d " % table[i][j][k],

        print

    print
    print



